# Reconnaissance de caractères et de symboles

## Détection de texte dans une image

Pour détecter le texte au sein d'une image, nous utilisons le plugin python **pytesseract**.

Dans un premier temps, nous effectuons des pré-traitements sur l'image (suppression de la couleur, dilatation, erosion pour réduire le bruit...) afin de la simplifier et d'accélerer le traitement.


Ensuite, nous extrayons (grâce à pytesseract) les zones contenant du texte puis, en bouclant sur ces zones, nous pouvons extraire les coordonnées et le contenu de ces dernières pour les stocker dans une liste réutilisable.

In [69]:
import cv2
import pytesseract
import ipywidgets as widgets
from IPython.display import Image, display
import threading
import numpy as np

In [70]:
def find_text(frame):
    img = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    kernel = np.ones((1, 1), np.uint8)
    img = cv2.dilate(img, kernel, iterations=10)
    img = cv2.erode(img, kernel, iterations=10)
    
    hImg, wImg = img.shape[:2]
    boxes = pytesseract.image_to_data(img)

    data = []
    
    for x, b in enumerate(boxes.splitlines( )):
        if x != 0:
            b = b.split( )
            if len(b)==12:
                x, y, w, h = int(b[6]), int(b[7]), int(b[8]), int(b[9])
                content = b[11]
                data.append((x, y, w, h, content))
                
    return data

## Détection de QR codes et de codes barres dans une image

Comme pour la détection du texte, les détections de codes barres et de codes QR peuvent être effectuées à l'aide d'un plugin python : **pyzbar**.

A partir d'une image légèrement prétraitée, nous pouvons utiliser la fonction **pyzbar.decode(...)** pour extraire les différents codes présents. Cette fonction est très pratique, et nous avons ajouté un comportement différent dans le cas où l'image (provenant d'une webcam par exemple) serait inversée. A ce moment là, nous allons décaler dans le sens opposé les zones détectées sur l'image par rapport au milieu de l'axe X.

In [72]:
import cv2
from pyzbar import pyzbar

In [73]:
def decode_image(img, frame_reversed=False):
    codes = pyzbar.decode(img)
    
    if frame_reversed:
        img_width = img.shape[1]
        reversed_codes = []

        for code in codes:
            reversed_x = img_width - code.rect.left - (code.rect.width / 2)

            reversed_rect = Rect(
                left=int(reversed_x - code.rect.width / 2),
                top=code.rect.top,
                width=code.rect.width,
                height=code.rect.height
            )

            reversed_code = Decoded(
                data=code.data,
                type=code.type,
                rect=reversed_rect,
                polygon=code.polygon,
                quality=code.quality,
                orientation=code.orientation
            )

            reversed_codes.append(reversed_code)

        return reversed_codes
    
    return codes

In [74]:
def update_image(img, code):
    (x, y, w, h) = code.rect
    cv2.rectangle(img, (x, y), (x + w, y + h), (0, 0, 255), 2)

    codeData = code.data.decode("utf-8")
    codeType = code.type

    text = "{} ({})".format(codeData, codeType)
    cv2.putText(img, text, (x, y - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 0, 255), 2)

    return img

In [75]:
def decode_and_update_image(original):
    if original is None:
        return [], original, original
    
    updated = original.copy()
    img = cv2.cvtColor(original, cv2.COLOR_BGR2GRAY)

    codes = decode_image(img)
    for code in codes:
        updated = update_image(updated, code)

    return codes, original, updated

In [76]:
def decode_and_update_image_from_path(img_path):
    original = cv2.imread(img_path)

    return decode_and_update_image(original)

## Détection de logos dans une image

La détection de logo est certainement la détection la plus compliquée du projet. Les ressources demandées sont très importantes alors il est nécessaire, en plus du pré-traitement de l'image, de la réduire en taille.

Pour comparer les frames à un ensemble de logos, nous allons utiliser l'algorithme d'intelligence artificielle **KNN (k nearest neighbor)** qui nous permet, à partir de la représentation d'un logo de trouver s'il est présent dans l'image.

Au départ, nous devons donc récupérer les **flanns** de tous les logos (ici ceux qui sont présents dans le dossier 'data/images/logos/src'). Les flanns sont les descripteurs du logo qui prennent en comptes les points-clés de l'image. Une fois ces flanns récupérés, nous pourrons les utiliser sur chaque frame pour tester la présence des logos grâce à la fonction **flann.knnMatch(...)**. 

Si la présence d'un logo est notée, il nous est ensuite possible de le retrouver la modification de perspective de ce logo au sein de notre frame grâce à la fonction **findHomography(...)** de **opencv**.

In [78]:
SIFT_SIGMA = 1.6
SIFT_INIT_SIGMA = 0.5
FX_FY = 0.7 # 0.5 = fast, 2 = slow

def resize_image(img, fx_fy):
    img = cv2.resize(img, (0, 0), fx=fx_fy, fy=fx_fy, interpolation=cv2.INTER_LINEAR)
    return img

def create_initial_image(img, sigma=SIFT_SIGMA):
    if len(img.shape) == 3 and img.shape[-1] == 3:
        img = cv2.cvtColor(img, cv2.COLOR_RGB2GRAY)

    sigma_diff = math.sqrt(max(sigma ** 2 - (2* SIFT_INIT_SIGMA) ** 2, 0.01))

    cv2.GaussianBlur(img, (0, 0), sigmaX=sigma_diff, sigmaY=sigma_diff)

    return img

In [79]:
def get_flanns(sift, logos_path):
    flanns = []
    
    index_logo = 1
    for img_path in os.listdir(logos_path):
        t1 = time.time()
        
        full_path = logos_path + "/" + img_path
        
        initial_logo = cv2.imread(full_path, cv2.IMREAD_GRAYSCALE)
        
        logo = create_initial_image(initial_logo)
        
        # find the keypoints and descriptors with SIFT 
        kp_image, desc_image = sift.detectAndCompute(logo, None) 

        # initializing the dictionary 
        index_params = dict(algorithm = 0, trees = 5) 
        search_params = dict() 

        # by using Flann Matcher 
        flann = cv2.FlannBasedMatcher(index_params, search_params)
        
        flanns.append((logo, kp_image, desc_image, flann))
        
        t2 = time.time()
        
        print(f'Récupération logo {index_logo} : {t2 - t1}')
        index_logo += 1
    
    return flanns

In [81]:
def find_logo_data(logos_id2name, logo_id, sift, flann, logo, desc_image, kp_image, original_image, scale_percent, distance_threshold=0.6):    
    resized_image = resize_image(original_image, scale_percent/100)
    image = create_initial_image(resized_image)
        
    # Find keypoints and descriptors in the current frame
    kp_grayframe, desc_grayframe = sift.detectAndCompute(image, None)

    # Find matches using FLANN
    matches = flann.knnMatch(desc_image, desc_grayframe, k=2)

    # Filter good points based on distance
    good_points = [m for m, n in matches if m.distance < distance_threshold * n.distance]

    if len(good_points) < 4:
        return None

    # Get corresponding points for perspective transformation
    query_pts = np.float32([kp_image[m.queryIdx].pt for m in good_points]).reshape(-1, 1, 2)
    train_pts = np.float32([kp_grayframe[m.trainIdx].pt for m in good_points]).reshape(-1, 1, 2)

    # Find perspective transformation
    matrix, mask = cv2.findHomography(query_pts, train_pts, cv2.RANSAC, 5.0, confidence=0.997)

    if matrix is None:
        return None
    
    # Apply perspective transformation to logo corners
    h, w = logo.shape
    pts = np.float32([[0, 0], [0, h], [w, h], [w, 0]]).reshape(-1, 1, 2)
    
    dst = cv2.perspectiveTransform(pts, matrix)

    dst /= scale_percent / 100.0
    
    return {
        'id': logo_id,
        'data': dst
    }

In [82]:
def get_logos_data(flanns, image, scale_percent):
    logos_data = []
    for index, (logo, kp_image, desc_image, flann) in enumerate(flanns):
        logos_data.append(find_logo_data(logos_id2name, index, sift, flann, logo, desc_image, kp_image, image, scale_percent))
    return logos_data

## Détection de texte, QR codes, codes barres et logos dans une image

Après avoir testé individuellement chacune de nos fonctions précédentes, nous pouvons fusionner le tout dans une seule grande partie qui s'occupe de récupérer les images de la webcam, analyser le texte, les codes et les logos, puis d'écrire les informations importantes directement sur l'image avant de l'afficher à l'écran.

On pourra noter ici l'instruction suivante dans la boucle de la fonction `view` :
```py
if index_frame % 12 == 0:
    data = find_text(original_frame)
```
qui nous permet de ne rechercher le texte approximativement une fois toutes les demi-secondes pour éviter la surcharge de travail de pytesseract.

De plus, nous pourrons noter ici dans la fonction `add_text_to_frame` :
```py
if len(content) < 5:
    continue
```
Cette instruction nous permet de n'afficher que les mots détectés d'une taille supérieure à 5. Cela évite d'afficher trop de mots à l'écran et se focalise sur les mots les plus longs que l'on pourrait supposer comme possédant une plus grande signification.

In [87]:
import cv2
import numpy as np
import os
import time
import math

import pytesseract
from pyzbar import pyzbar
from pyzbar.pyzbar import Point, Rect, Decoded

import ipywidgets as widgets
from IPython.display import Image, display
import threading

In [95]:
LOGOS_PATH = './data/images/logos/sources'
OUTPUT_PATH = './data/outputs'

sift = cv2.SIFT_create()

logos_id2name = {}
for index, img_path in enumerate(os.listdir(LOGOS_PATH)):
    logos_id2name[index] = img_path.split('.')[0]
print(logos_id2name)

t1 = time.time()
flanns = get_flanns(sift, LOGOS_PATH)
t2 = time.time()
print(f"Flanns récupérés ({t2 - t1}s)")

{0: 'la-parisienne', 1: 'lissac', 2: 'folio-essais', 3: 'traditional-medicinals'}
Récupération logo 1 : 0.026321887969970703
Récupération logo 2 : 0.02335524559020996
Récupération logo 3 : 0.02161407470703125
Récupération logo 4 : 0.01456308364868164
Flanns récupérés (0.08712172508239746s)


In [91]:
def add_text_to_frame(data, frame):
    for (x, y, w, h, content) in data:
        if len(content) < 5:
            continue
        cv2.rectangle(frame, (x, y), (w+x, h+y), (0, 255, 0), 1)
        cv2.putText(frame, content, (x, y), cv2.FONT_HERSHEY_COMPLEX, 0.5, (0, 255, 0), 2)
    
    return frame

def add_codes_to_frame(codes, frame):
    for code in codes:
        frame = update_image(frame, code)
    return frame

def add_logos_to_frame(logos_data, frame):
    if logos_data is None:
        return frame
    
    for logo_data in logos_data:
        if logo_data is None:
            continue
            
        logo_name = logos_id2name.get(logo_data['id'], "Unknown")
        data = logo_data['data']
        
        # Draw the perspective-transformed logo on the frame
        homography = cv2.polylines(frame, [np.int32(data)], True, (255, 0, 0), 3)

        # Write the name of the logo on the image
        font = cv2.FONT_HERSHEY_SIMPLEX
        font_scale = 0.5
        font_thickness = 1
        text_size = cv2.getTextSize(logo_name, font, font_scale, font_thickness)[0]
        text_position = (int(data[0, 0, 0] - text_size[0] / 2), int(data[0, 0, 1] - 5))
        cv2.putText(homography, logo_name, text_position, font, font_scale, (255, 0, 0), font_thickness, cv2.LINE_AA)
    
    return frame
        

In [125]:
stopButton = widgets.ToggleButton(
    value=False,
    description='Stop',
    disabled=False,
    button_style='danger',
    tooltip='Description',
    icon='square'
)


def view(button):
    previous_logo_id = -1
    previous_codes = None
    
    cap = cv2.VideoCapture(0)
    display_handle=display(None, display_id=True)
    
    index_frame = 0
    data = []
    
    

    fourcc = cv2.VideoWriter_fourcc(*'MJPG')
    videoWriter = cv2.VideoWriter(OUTPUT_PATH + '/presentation.mp4', fourcc, 8.0, (640,480))

    
    while True:
        index_frame += 1
            
        if stopButton.value==True:
            cap.release()
            videoWriter.release()
            display_handle.update(None)
        
        _, original_frame = cap.read()
            
       
        if index_frame % 12 == 0:
            data = find_text(original_frame)
        
        
        barcode_frame = cv2.flip(original_frame, 1) # camera reverses image
        codes = decode_image(barcode_frame, frame_reversed=True)
        logos_data = get_logos_data(flanns, original_frame, scale_percent=70)
        
            
        # Modify current frame
        frame = add_text_to_frame(data, original_frame)
        frame = add_codes_to_frame(codes, frame)
        frame = add_logos_to_frame(logos_data, frame)
        
        videoWriter.write(frame)
            
        _, frame = cv2.imencode('.jpeg', frame)
        
        
        display_handle.update(Image(data=frame.tobytes()))
        
        


display(stopButton)
thread = threading.Thread(target=view, args=(stopButton,))
thread.start()

Widget Javascript not detected.  It may not be installed or enabled properly. Reconnecting the current kernel may help.


None

OpenCV: FFMPEG: tag 0x47504a4d/'MJPG' is not supported with codec id 7 and format 'mp4 / MP4 (MPEG-4 Part 14)'
OpenCV: FFMPEG: fallback to use tag 0x7634706d/'mp4v'
Exception in thread Thread-50 (view):
Traceback (most recent call last):
  File "/usr/lib/python3.10/threading.py", line 1016, in _bootstrap_inner
    self.run()
  File "/usr/lib/python3.10/threading.py", line 953, in run
    self._target(*self._args, **self._kwargs)
  File "/tmp/ipykernel_24228/168502909.py", line 43, in view
  File "/tmp/ipykernel_24228/463870029.py", line 2, in decode_image
  File "/home/antoine/.local/lib/python3.10/site-packages/pyzbar/pyzbar.py", line 207, in decode
    pixels, width, height = _pixel_data(image)
  File "/home/antoine/.local/lib/python3.10/site-packages/pyzbar/pyzbar.py", line 173, in _pixel_data
    pixels, width, height = image
TypeError: cannot unpack non-iterable NoneType object


Aujourd'hui, de plus en plus d'outils nous permettent d'extraire des informations des vidéos de façon extrêmement rapide. Le pré-traitement et la réduction d'échelle d'une image sont essentiels pour une analyse efficace en temps réel.

Notre projet peut être perçu comme un **POC (*proof-of-concept*)**. Il reste des corrections à apporter, notamment sur la détection de textes ou sur la détection de logos qui reste délicate quand un logo entre ou sort de l'écran. Nous pourrions adopter des comportements différents quand un logo ou un code-barre est detecté ou nous pourrions même prioriser certaines informations à l'écran quand ce dernier est surchargé.